In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series


Demolition_Permits = pd.read_csv('./data/_dcebfb2135a2bf5a6392493bd61aba22_detroit-demolition-permits.tsv',  dtype={'site_location': str}, 
                                 sep='\t', low_memory=False )
Demolition_Permits

,PERMIT_NO,PERMIT_APPLIED,PERMIT_ISSUED,PERMIT_EXPIRES,SITE_ADDRESS,BETWEEN1,PARCEL_NO,LOT_NUMBER,SUBDIVISION,CASE_TYPE,...,CONTRACTOR_ADDRESS1,CONTRACTOR_ADDRESS2,CONTRACTOR_CITY,CONTRACTOR_STATE,CONTRACTOR_ZIP,CONDITION_FOR_APPROVAL,site_location,owner_location,contractor_location,geom
0,BLD2015-03955,8/28/15,8/28/15,NaN,4331 BARHAM,BETWEEN VOIGHT AND WAVENEY,21 65525-6,S15,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"4331 BARHAM\nDetroit, MI\n(42.394106, -82.9474)","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
1,BLD2015-04083,8/28/15,8/28/15,NaN,9707 BESSEMORE,BETWEEN VINTON AND GRATIOT,19 2312.,40,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"9707 BESSEMORE\nDetroit, MI\n(42.395122, -83.0...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
2,BLD2015-03976,8/28/15,8/28/15,NaN,5315 BERKSHIRE,BETWEEN SOUTHAMPTON AND FRANKFORT,21 69321.,445,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5315 BERKSHIRE\nDetroit, MI\n(42.40322, -82.94...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
3,BLD2015-03781,8/28/15,8/28/15,NaN,16670 BRINGARD DR,BETWEEN CUSHING AND SHAKESPEARE,21 25930.,97,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"16670 BRINGARD DR\nDetroit, MI\n(42.44664, -82...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
4,BLD2015-03677,8/28/15,8/28/15,NaN,1454 BEATRICE,BETWEEN TORONTO AND GILROY,20 12871.012,21,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"1454 BEATRICE\nDetroit, MI\n(42.276956, -83.14...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
5,BLD2015-03914,8/28/15,8/28/15,NaN,5035 BALFOUR,BETWEEN FRANKFORT AND WARREN,21 68774.,163,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5035 BALFOUR\nDetroit, MI\n(42.400373, -82.94705)","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
6,BLD2015-03950,8/28/15,8/28/15,NaN,5297 BALFOUR,BETWEEN SOUTHAMPTON AND FRANKFORT,21 68751.,161,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5297 BALFOUR\nDetroit, MI\n(42.402349, -82.948...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
7,BLD2015-03953,8/28/15,8/28/15,NaN,5315 BALFOUR,BETWEEN SOUTHAMPTON AND FRANKFORT,21 68749.,160,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5315 BALFOUR\nDetroit, MI\n(42.402497, -82.948...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
8,BLD2015-03945,8/28/15,8/28/15,NaN,5280 BALFOUR,BETWEEN VAN BUREN AND CONSTANCE,22 75852.,S10,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5280 BALFOUR\nDetroit, MI\n(42.402275, -82.948...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
9,BLD2015-03821,8/28/15,8/28/15,NaN,427 ASHLAND,BETWEEN ESSEX AND AVONDALE,21 62539.,235,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"427 ASHLAND\nDetroit, MI\n(42.366963, -82.935172)","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN


In [2]:
import dateutil.parser
from datetime import date

def dateParser(currdate):
    try:
        return dateutil.parser.parse(currdate)
    except:
        return None

In [13]:
dateParser("8/28/15")

datetime.datetime(2015, 8, 28, 0, 0)

In [3]:
import sqlite3


def Insert(row, idx, cur):  
    name = 'Demolition{}'.format(idx+1)
    cur.execute("select Id from LocationReferenced where OriginId='{}'".format(name))
    idx = cur.fetchone()
    if idx is None:
        return
    index = idx[0]
    date = dateParser(row['PERMIT_APPLIED'])
    if date is None:
        return
    info ={'index':index, 'date':date}
    cur.execute('insert into DemolishedData (LocationReferencedId, Date) values (:index,:date)', info)

con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()

for index, row in Demolition_Permits.iterrows():
    Insert(row, index, cur)
con.commit()
con.close()